In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
# import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import utils, plot, cosmo
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

## Create SAM & Realization

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.sam.MMBulge_Simple()     # M-MBulge Relation            (MMB)

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)

In [ ]:
sam = holo.sam.Semi_Analytic_Model()
fobs = utils.nyquist_freqs(20.0*YR, 0.1*YR)
# fobs = utils.nyquist_freqs(200.0*YR, 1.0*YR)

In [ ]:
gff, gwf, gwb = holo.sam.sampled_gws_from_sam(
    sam, fobs, sample_threshold=10.0,
    cut_below_mass=1e7*MSOL, limit_merger_time=4*GYR
)

In [ ]:
def draw_sampled_gwb(ax, fobs, gff, gwf, gwb):
    xx = kale.utils.midpoints(fobs) * YR   # [1/sec] ==> [1/yr]
    col, = ax.plot(xx, gwb, ls='-', alpha=0.75, lw=0.75)
    col = col.get_color()

    idx = (gwf > gwb)
    xx = gff * YR   # [1/sec] ==> [1/yr]
    ax.scatter(xx[idx], gwf[idx], color=col, s=10, alpha=0.5)
    ax.scatter(xx[~idx], gwf[~idx], edgecolor=col, facecolor='none', s=10, alpha=0.5)
    return


fig, ax = plot.figax()

amp = 10e-16
xx = utils.minmax(fobs) * YR
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)

draw_sampled_gwb(ax, fobs, gff, gwf, gwb)
plt.show()

## Plot Realizations

In [ ]:
import glob
import os
path = holo.__file__
path = os.path.join(os.path.dirname(path), os.path.pardir, 'output', '')
path = os.path.abspath(path)
pattern = os.path.join(path, '*.npz')

files = glob.glob(pattern)

print(path, pattern)
print(f"Found {len(files)} files.")
if len(files) > 0:
    print(f"\te.g. '{files[0]}'")

fig, ax = plot.figax()

amp = 10e-16
xx = utils.minmax(fobs) * YR
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)

for ff in files:
    data = np.load(ff)
    draw_sampled_gwb(ax, data['fobs'], data['gff'], data['gwf'], data['gwb'])

plt.show()
